### Aggressive Language Detection

The following notebook outlines the steps conducted to detect toxic discourse across the Reddit comments. To identify toxic language, we utilize a model known as Detoxify that gives a probability score of the toxicity in the Reddit comment. There are six different toxicity categories.

-------------
- **Step 1**: Read the data.
- **Step 2.** Compute the aggressive language variables.

Install libraries

In [1]:
pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 55.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np 
import pandas as pd

## Step 1: 
Read the data. <br>
We consider a few weeks of data as an illustrative example. In the main cross-sectional analyses, this same exercise was conducted for each month from January through June. For simplicity, we demonstrate the exercise with just a subset of weeks from May and June.

In [3]:
#Step 1: Read the data
data_May_final = pd.read_parquet('data/MayJune_processedData/data_May_final.parquet')
data_June_final = pd.read_parquet('data/MayJune_processedData/data_June_final.parquet')
print(len(data_May_final))
print(len(data_June_final))

4928924
4736165


Display the different subreddits in the data.

In [8]:
print(data_May_final['subreddit'].value_counts())
print(data_June_final['subreddit'].value_counts())

amitheasshole    1681047
worldnews         991545
politics          952785
news              526745
science           116348
Name: subreddit, dtype: int64
amitheasshole    1629128
politics         1069313
worldnews         758924
news              564417
science            97752
Name: subreddit, dtype: int64


Install the TQDM library so that we can track progress of analysis.

In [11]:
pip install tqdm


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from tqdm import tqdm
tqdm.pandas()

Now we want to create a column known as Week. Essentially, our entire dataset consists of 26 weeks. Since we are considering the months of May and June in this example, we need labels from Week 18 to Week 22. We create a function to assign the week number based on which week the data point falls in.

In [13]:
def week_dummy(record): #week 9 to week 17 were in march and april 2022
    if record['date'] in {'2022-05-01','2022-05-02','2022-05-03','2022-05-04','2022-05-05','2022-05-06','2022-05-07'}:
          return 'week18'
    if record['date'] in {'2022-05-08','2022-05-09','2022-05-10','2022-05-11','2022-05-12','2022-05-13','2022-05-14'}:
          return 'week19'
    if record['date'] in {'2022-05-15','2022-05-16','2022-05-17','2022-05-18','2022-05-19','2022-05-20','2022-05-21'}:
          return 'week20'
    if record['date'] in {'2022-05-22','2022-05-23','2022-05-24','2022-05-25','2022-05-26','2022-05-27','2022-05-28'}:
          return 'week21'  
    if record['date'] in {'2022-05-29','2022-05-30','2022-05-31','2022-06-01','2022-06-02','2022-06-03','2022-06-04'}:
          return 'week22'
    if record['date'] in {'2022-06-05','2022-06-06','2022-06-07','2022-06-08','2022-06-09','2022-06-10','2022-06-11'}:
          return 'week23'
    if record['date'] in {'2022-06-12','2022-06-13','2022-06-14','2022-06-15','2022-06-16','2022-06-17','2022-06-18'}:
          return 'week24'
    if record['date'] in {'2022-06-19','2022-06-20','2022-06-21','2022-06-22','2022-06-23','2022-06-24','2022-06-25'}:
          return 'week25'
    if record['date'] in {'2022-06-26','2022-06-27','2022-06-28','2022-06-29','2022-06-30'}:
          return 'week26'
    return 'incorrect' 

#Now create week data
data_May_final['week'] = data_May_final.progress_apply(week_dummy, axis=1)
data_June_final['week'] = data_June_final.progress_apply(week_dummy, axis=1)
print(data_May_final['week'].value_counts())
print(data_June_final['week'].value_counts())

week18    1008518
week21     988656
week19     950801
week20     927040
week22     393455
Name: week, dtype: int64
week25    1066216
week24     919563
week23     881908
week26     803544
week22     448303
Name: week, dtype: int64


100%|██████████| 4119534/4119534 [01:58<00:00, 34681.93it/s]


In [14]:
MayJune_w18 = data_May_final[data_May_final['week'] == 'week18']
MayJune_w19 = data_May_final[data_May_final['week'] == 'week19']
MayJune_w20 = data_May_final[data_May_final['week'] == 'week20']
MayJune_w21 = data_May_final[data_May_final['week'] == 'week21']
MayJune_w22_part1 = data_May_final[data_May_final['week'] == 'week22']
MayJune_w22_part2 = data_June_final[data_June_final['week'] == 'week22']
MayJune_w22 = pd.concat([MayJune_w22_part1, MayJune_w22_part2], ignore_index=True, axis=0)
MayJune_w23 = data_June_final[data_June_final['week'] == 'week23']
MayJune_w24 = data_June_final[data_June_final['week'] == 'week24']
MayJune_w25 = data_June_final[data_June_final['week'] == 'week25']
MayJune_w26 = data_June_final[data_June_final['week'] == 'week26']

print(len(MayJune_w18))
print(len(MayJune_w19))
print(len(MayJune_w20))
print(len(MayJune_w21))
print(len(MayJune_w22))
print(len(MayJune_w23))
print(len(MayJune_w24))
print(len(MayJune_w25))
print(len(MayJune_w26))

1008518
950801
927040
988656
841758
881908
919563
1066216
803544


In [15]:
MayJune_w18.to_parquet('data/weeklyData/MayJune_w18')
MayJune_w19.to_parquet('data/weeklyData/MayJune_w19')
MayJune_w20.to_parquet('data/weeklyData/MayJune_w20')
MayJune_w21.to_parquet('data/weeklyData/MayJune_w21')
MayJune_w22.to_parquet('data/weeklyData/MayJune_w22')
MayJune_w23.to_parquet('data/weeklyData/MayJune_w23')
MayJune_w24.to_parquet('data/weeklyData/MayJune_w24')
MayJune_w25.to_parquet('data/weeklyData/MayJune_w25')
MayJune_w26.to_parquet('data/weeklyData/MayJune_w26')

Thus, at this stage, we have prepared our data at the weekly level and are ready for the next step of the analysis.

## **Step 2.** 
Compute the aggressive language variables. <br>

In Step 2, we compute the level of toxic discourse in each Reddit comment. We use the Detoxify LLM to assign a toxicity score to every comment. First, we need to install the Detoxify package.

In [3]:
pip install detoxify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from detoxify import Detoxify

In [5]:
detoxify_model = Detoxify(
    model_type='original',
    device='cuda'
)

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /home/datalore/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
100%|██████████| 418M/418M [00:09<00:00, 48.5MB/s]


Run an example to check whether we are able to call the model and make a prediction. As shown below, the prediction is essentially a dictionary of different toxicity scores based on the categories. We will now use the same way of calling the Detoxify model on every comment in the Reddit data set and obtain a dictionary for every comment.

In [6]:
predicts_dict = detoxify_model.predict("this is a sample sentence where is is no insulting or toxic behavior")
print(len(predicts_dict))
print(type(predicts_dict))
print(predicts_dict)
print(predicts_dict['threat'])

6
<class 'dict'>
{'toxicity': 0.0024756158, 'severe_toxicity': 8.681209e-05, 'obscene': 0.00023586965, 'threat': 0.00010463204, 'insult': 0.0002232705, 'identity_attack': 0.00015557937}
0.00010463204


We then use this function to compute toxicity scores for the entire dataset. For illustration, we show a subset of data from a single week. In this example, we use a simple lambda function to calculate the toxicity (or aggressive language) dictionary for each comment and save the results.

### Process week 18.

In [7]:
MayJune_w18 = pd.read_parquet('data/weeklyData/MayJune_w18')
print(len(MayJune_w18))

1008518


In [8]:
#start with week18
MayJune_w18_part1 = MayJune_w18[MayJune_w18['date'].isin(['2022-05-01','2022-05-02','2022-05-03'])]
MayJune_w18_part2 = MayJune_w18[MayJune_w18['date'].isin(['2022-05-04','2022-05-05','2022-05-06','2022-05-07'])]

print(len(MayJune_w18_part1))
print(len(MayJune_w18_part2))

450203
558315


In [17]:
#running the model on one row
detoxify_model.predict(MayJune_w18_part1['body'].iloc[0])

{'toxicity': 0.0013658818,
 'severe_toxicity': 9.632462e-05,
 'obscene': 0.00017382219,
 'threat': 0.00012454727,
 'insult': 0.0001976232,
 'identity_attack': 0.00015838591}

In [18]:
#week 18 part 1
MayJune_w18_part1['aggLangDict'] = MayJune_w18_part1['body'].progress_apply( lambda x: detoxify_model.predict(x))

100%|██████████| 450203/450203 [1:26:41<00:00, 86.55it/s]
<ipython-input-18-d716995a1af7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MayJune_w18_part1['aggLangDict'] = MayJune_w18_part1['body'].progress_apply( lambda x: detoxify_model.predict(x))


In [20]:
MayJune_w18_part1.to_parquet('data/weeklyAggressiveLang/MayJune_w18_part1')